In [2]:
import numpy as np
import matplotlib.pyplot as plt
from nilearn import datasets
from nilearn import connectome
from nilearn import plotting
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import json
from datetime import datetime

import utils as ut 
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
# SVM classifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

atlas = datasets.fetch_atlas_msdl()
atlas_filename = atlas["maps"]
labels = atlas["labels"]
num_ROIs = len(labels)

%matplotlib inline

In [ ]:
# if correlation matrice not yet computed
data_path = "/Users/abelrassat/Desktop/NSC/Complex_Networks_Theory_and_Application/Project/" # change to your path where to save the fMRI data
site_ids = ["NYU"]
age_range = (0, 100)
save_name = "NYU_0-100"
# may need to run this cell several times to resume downloading if connection is lost
ut.fmri_data_preparation(data_path, site_ids, age_range, proportional_threshold=0.3, verbose=True, save_name=save_name)

In [3]:
# load data
X = np.load("X_NYU_0-100.npy")
y = np.load("y_NYU_0-100.npy")

In [4]:
# compute independent pagerank in advance for all subjects to save time
r_pagerank = 0.85
Z_indep, y, not_converged = ut.independent_pagerank(X, y, num_ROIs, labels, r_pagerank=r_pagerank, save_name="Z_indep_NYUf.npy", show_best_features=False)

Independent centrality measures:   0%|          | 0/172 [00:00<?, ?it/s]

/Users/abelrassat/opt/anaconda3/envs/cn_final_proj/lib/python3.11/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/Users/abelrassat/opt/anaconda3/envs/cn_final_proj/lib/python3.11/site-packages/networkx/algorithms/link_analysis/pagerank_alg.py:496: RuntimeWarning: invalid value encountered in subtract
  err = np.absolute(x - xlast).sum()


Subjects for which Page Rank did not converge: [159] 
 Delete them manually from the dataset.


In [5]:
# For now remove the not converged subjects
X = np.delete(X, not_converged, axis=0)
X_ASD = X[y==1]
X_TD = X[y==0]
print("Number of ASD subjects: ", X_ASD.shape[0])
print("Number of TD subjects: ", X_TD.shape[0])

Number of ASD subjects:  74
Number of TD subjects:  97


In [6]:
# hyperparameters
n_splits = 10
inter_layer_edges_weight = 0.3
interconnection_per_layer_pair = 20
r_pagerank = 0.85
num_features = 30

seed = 0 # for reproducibility

print(X.shape, y.shape)

# Initialize StratifiedKFold with the same splits for both representations
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=seed)

scores_mult = np.empty([0], dtype=float)
scores_indep = np.empty([0], dtype=float)
actual_labels = np.empty([0], dtype=int)
predicted_labels_mult = np.empty([0], dtype=int)
predicted_labels_indep = np.empty([0], dtype=int)
p_values_mult = np.empty([0], dtype=float)
p_values_indep = np.empty([0], dtype=float)

# Perform leave-one-out cross-validation
for train_index, test_index in tqdm(kfold.split(X, y), desc="Cross-validation", total=kfold.get_n_splits()):
    # compute the representation on the training set
    X_train = X[train_index]
    Z_train_indep, Z_test = Z_indep[train_index], Z_indep[test_index]
    y_train, y_test = y[train_index], y[test_index]
    X_train_ASD, X_train_TC = X_train[np.where(y_train == 1)], X_train[np.where(y_train == 0)]
    X_train_TC = X_train_TC[:X_train_ASD.shape[0]] # if want to have equal number of patients and controls

    # method 1: compute the representation on the training set with multiplex networks
    ut.multiplex_pagerank(X_train_ASD, num_ROIs, labels, inter_layer_edges_weight=inter_layer_edges_weight, interconnection_per_layer_pair=interconnection_per_layer_pair, save_name_pagerank="Z_multiplex_ASD_NYU_kf_final.npy", show_best_features=False)
    ut.multiplex_pagerank(X_train_TC, num_ROIs, labels, inter_layer_edges_weight=inter_layer_edges_weight, interconnection_per_layer_pair=interconnection_per_layer_pair, save_name_pagerank="Z_multiplex_TC_NYU_kf_final.npy", show_best_features=False)
    

    # separate the representations for the two groups to find best features
    Z_train_ASD_mult = np.load("Z_multiplex_ASD_NYU_kf_final.npy")
    Z_train_TC_mult = np.load("Z_multiplex_TC_NYU_kf_final.npy")
    Z_train_indep = Z_indep[train_index]
    Z_train_ASD_indep = Z_train_indep[np.where(y_train == 1)]
    Z_train_TC_indep = Z_train_indep[np.where(y_train == 0)]
    

    # select best features for each representation and normalize the data
    Z_train_ASD_mult, Z_train_TC_mult, Z_test_mult, pval_mult = ut.feature_processing(Z_train_ASD_mult, Z_train_TC_mult, Z_test, num_ROIs, labels, num_features, show_best_features=False, independent=False)
    Z_train_ASD_indep, Z_train_TC_indep, Z_test_indep, pval_indep = ut.feature_processing(Z_train_ASD_indep, Z_train_TC_indep, Z_test, num_ROIs, labels, num_features, show_best_features=False, independent=True)                                                                 

    Z_train_mult = np.concatenate((Z_train_ASD_mult, Z_train_TC_mult), axis=0)
    Z_train_indep = np.concatenate((Z_train_ASD_indep, Z_train_TC_indep), axis=0)
    y_train_sep_mult = np.concatenate((np.ones(Z_train_ASD_mult.shape[0]), np.zeros(Z_train_TC_mult.shape[0])), axis=0)
    y_train_sep_indep = np.concatenate((np.ones(Z_train_ASD_indep.shape[0]), np.zeros(Z_train_TC_indep.shape[0])), axis=0)
    

    # 1. Train and evaluate the classifier on the multiplex representation
    classifier_mult = svm.SVC(kernel='rbf', C=50, probability=True, random_state=seed)
    classifier_mult.fit(Z_train_mult, y_train_sep_mult)
    score_mult = classifier_mult.score(Z_test_mult, y_test)
    scores_mult = np.append(scores_mult, score_mult)

    # 2. Train and evaluate the classifier on the independent networks representation
    classifier_indep = svm.SVC(kernel='rbf', C=10, probability=True, random_state=seed)
    classifier_indep.fit(Z_train_indep, y_train_sep_indep)
    score_indep = classifier_indep.score(Z_test_indep, y_test)
    scores_indep = np.append(scores_indep, score_indep)

    actual_labels = np.append(actual_labels, y_test)
    predicted_labels_mult = np.append(predicted_labels_mult, classifier_mult.predict(Z_test_mult))
    predicted_labels_indep = np.append(predicted_labels_indep, classifier_indep.predict(Z_test_indep))
    p_values_mult = np.append(p_values_mult, pval_mult)
    p_values_indep = np.append(p_values_indep, pval_indep)

    # Visualisation options
    # ut.debug_tool_plot_2d(Z_train_mult, y_train_sep_mult, Z_test_mult, y_test)
    # ut.debug_tool_plot_2d(Z_train_indep, y_train_sep_indep, Z_test_indep, y_test)
    # ut.predict_proba(classifier_mult, Z_test_mult, y_test, show_plot=True)
    # ut.predict_proba(classifier_indep, Z_test_indep, y_test, show_plot=True)

    print(f"Scores up to now: \n Multiplex: {scores_mult} \n Running mean {np.mean(scores_mult)} \n Independent: {scores_indep} \n Running mean {np.mean(scores_indep)}")

# Print the mean accuracy for each representation
print(f"Mean Accuracy: \n Multiplex: {np.mean(scores_mult)} \n Independent: {np.mean(scores_indep)}")

(171, 39, 39) (171,)


Cross-validation:   0%|          | 0/10 [00:00<?, ?it/s]

Building supra-adjacency matrix:   0%|          | 0/67 [00:00<?, ?it/s]

Negative entries of leading eigenvector


Building supra-adjacency matrix:   0%|          | 0/67 [00:00<?, ?it/s]

...

Negative entries of leading eigenvector
Scores up to now: 
 Multiplex: [0.61111111 0.52941176 0.58823529 0.76470588 0.70588235 0.58823529
 0.88235294 0.35294118 0.52941176] 
 Running mean 0.6169208424110385 
 Independent: [0.5        0.58823529 0.35294118 0.88235294 0.64705882 0.41176471
 0.52941176 0.52941176 0.64705882] 
 Running mean 0.5653594771241831


Building supra-adjacency matrix:   0%|          | 0/65 [00:00<?, ?it/s]

Negative entries of leading eigenvector


Building supra-adjacency matrix:   0%|          | 0/65 [00:00<?, ?it/s]

Scores up to now: 
 Multiplex: [0.61111111 0.52941176 0.58823529 0.76470588 0.70588235 0.58823529
 0.88235294 0.35294118 0.52941176 0.64705882] 
 Running mean 0.6199346405228758 
 Independent: [0.5        0.58823529 0.35294118 0.88235294 0.64705882 0.41176471
 0.52941176 0.52941176 0.64705882 0.35294118] 
 Running mean 0.5441176470588236
Mean Accuracy: 
 Multiplex: 0.6199346405228758 
 Independent: 0.5441176470588236


In [11]:
# Save confusion matrices
conf_matrix_mult = confusion_matrix(actual_labels, predicted_labels_mult)
conf_matrix_indep = confusion_matrix(actual_labels, predicted_labels_indep)

In [9]:
dict = {'scores_mult': scores_mult.tolist(), 'scores_indep': scores_indep.tolist(), 'p_values_mult': p_values_mult.tolist(), 'p_values_indep': p_values_indep.tolist(), "conf_matrix_mult": conf_matrix_mult.tolist(), "conf_matrix_indep": conf_matrix_indep.tolist()}

# Get the current date and time
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Specify the file path
file_path = f'Results_{num_features}_features_{timestamp}.json'
print(file_path)

# Save the dictionary as a JSON file
with open(file_path, 'w') as json_file:
    json.dump(dict, json_file)

Results_30_features_20240128_201902.json
